# VectorDB Question Ansering with Sources

This notebook goes over how to do question-answering with sources. It does this in a few different ways - first showing how you can use the `QAWithSourcesChain` to take in documents and use those, and next showing the `VectorDBQAWithSourcesChain`, which also does the lookup of the documents from a vector database. 

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores.faiss import FAISS

In [2]:
with open('../state_of_the_union.txt') as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings()

In [3]:
docsearch = FAISS.from_texts(texts, embeddings)

In [4]:
# Add in a fake source information
for i, d in enumerate(docsearch.docstore._dict.values()):
    d.metadata = {'source': f"{i}-pl"}

### VectorDBQAWithSourcesChain

This shows how to use the `VectorDBQAWithSourcesChain`, which uses a vector database to look up relevant documents.

In [9]:
from langchain.chains import VectorDBQAWithSourcesChain

In [10]:
chain = VectorDBQAWithSourcesChain.from_llm(OpenAI(temperature=0), vectorstore=docsearch)

In [11]:
chain({"question": "What did the president say about Justice Breyer"}, return_only_outputs=True)

{'answer': ' The president thanked Justice Breyer for his service.',
 'sources': '27-pl'}